In [1]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_chunks_unite_instances2
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import matplotlib.pyplot as plt 
import shutil

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[(0.267004, 0.004874, 0.329415), (0.277018, 0.050344, 0.375715), (0.282656, 0.100196, 0.42216), (0.28229, 0.145912, 0.46151), (0.276194, 0.190074, 0.493001), (0.265145, 0.232956, 0.516599), (0.252194, 0.269783, 0.531579), (0.235526, 0.309527, 0.542944), (0.21813, 0.347432, 0.550038), (0.201239, 0.38367, 0.554294), (0.185556, 0.41857, 0.556753), (0.171176, 0.45253, 0.557965), (0.159194, 0.482237, 0.558073), (0.14618, 0.515413, 0.556823), (0.133743, 0.548535, 0.553541), (0.123463, 0.581687, 0.547445), (0.119483, 0.614817, 0.537692), (0.128087, 0.647749, 0.523491), (0.150148, 0.676631, 0.506589), (0.19109, 0.708366, 0.482284), (0.24607, 0.73891, 0.452024), (0.311925, 0.767822, 0.415586), (0.386433, 0.794644, 0.372886), (0.468053, 0.818921, 0.323998), (0.545524, 0.838039, 0.275626), (0.636902, 0.856542, 0.21662), (0.730889,

Here we define the dataset depending on kitti sequence!

In [2]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
end_inds = {0:4541,1:1100,2:4661,3:800,4:271,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 7
old = False 
if SEQUENCE_NUM == 7 : 
        old = True 
ind_start = 0
ind_end = end_inds[SEQUENCE_NUM]
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = 'pcd_preprocessed/output_chunks/'

out_folder_ncuts = out_chunks + 'test_data' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_folder_ncuts):
        shutil.rmtree(out_folder_ncuts)
os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [3]:
if os.path.exists(out_folder + 'all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)

This cell can be ignored after first run as outputs are stored 

In [4]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_nonground_minor,kitti_labels['seg_nonground'])])
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz',
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )
        o3d.visualization.draw_geometries([pcd_nonground_minor])


In [5]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd')
print(pcd_ground_minor)
kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz') as data :
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

PointCloud with 11580178 points.


In [6]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

"\npcd_new = o3d.geometry.PointCloud()\npts_num = 1000000\npcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])\n\nmap_labelled = color_pcd_by_labels(pcd_new,                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))\n\no3d.visualization.draw_geometries([map_labelled])\n#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)\n"

Now we subsample the poses based on a voxel_size

In [7]:
if os.path.exists(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz') == False : 
	print(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz')
	poses, positions, \
	sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start,sequence_num=SEQUENCE_NUM,out_folder=out_folder)



In [8]:
with np.load(f'{out_folder}subsampled_data{SEQUENCE_NUM}_0.npz') as data:
	poses=data['poses']
	positions=data['positions']
	sampled_indices_local = data['sampled_indices_local']
	sampled_indices_global=data['sampled_indices_global']

Now we can split the point cloud into chunks based on a tbd chunk_size

In [9]:
overlap = 3
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

Downsampled from (541161, 3) to (7776, 3) points (non-ground)
Downsampled from (243917, 3) to (5294, 3) points (ground)
Downsampled from (454490, 3) to (7216, 3) points (non-ground)
Downsampled from (262770, 3) to (4499, 3) points (ground)
Downsampled from (479361, 3) to (7341, 3) points (non-ground)
Downsampled from (304983, 3) to (6061, 3) points (ground)
Downsampled from (479273, 3) to (6272, 3) points (non-ground)
Downsampled from (246003, 3) to (4418, 3) points (ground)
Downsampled from (473079, 3) to (8385, 3) points (non-ground)
Downsampled from (221993, 3) to (4446, 3) points (ground)
Downsampled from (488393, 3) to (9063, 3) points (non-ground)
Downsampled from (220415, 3) to (5078, 3) points (ground)
Downsampled from (545707, 3) to (8986, 3) points (non-ground)
Downsampled from (292301, 3) to (4209, 3) points (ground)
Downsampled from (409731, 3) to (6703, 3) points (non-ground)
Downsampled from (441195, 3) to (4976, 3) points (ground)
Downsampled from (300320, 3) to (4742, 3

In [10]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None,semantics=False):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])
    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 and semantics == False : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
    if semantics : 
        pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors)
    else : 
        pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/255)
    return pcd_colored

In [11]:
def find_index_2d_list(my_list, target_sublist):
    for i, sublist in enumerate(my_list):
        if sublist == list(target_sublist): 
            return i
    return None  # Sublist not found in the list


In [12]:
def get_labels_from_cols(colors,labels,pcd):
	cur_cols = np.asarray(pcd.colors)
	unique_labels = list(np.unique(labels))
	new_labels = np.zeros_like(labels)
	for label in unique_labels: 
		idcs = np.where(labels == label)[0]
		col = cur_cols[idcs[0]]
		idx = find_index_2d_list(colors,col)
		new_labels[idcs] = idx 
	return new_labels
	
def normalize_vector(v):
    """
    Normalize a 2D vector to have unit norm.

    Parameters:
    - v: A tuple or list representing the 2D vector (x, y).

    Returns:
    - A tuple representing the normalized 2D vector.
    """
    norm = np.sqrt(v[0]**2 + v[1]**2)
    return np.array([v[0] / norm, v[1] / norm])

def find_unit_norm_orthogonal_vectors(v):
    """
    Find two orthogonal vectors to a given 2D vector v, both of unit norm.

    Parameters:
    - v: A tuple or list representing the 2D vector (x, y).

    Returns:
    - A tuple containing two tuples, each representing a unit norm orthogonal vector.
    """
    # Given vector v = (x, y)
    x, y = v

    # Calculate orthogonal vectors
    v_perp1 = (y, -x)
    v_perp2 = (-y, x)

    # Normalize the orthogonal vectors
    v_perp1_normalized = normalize_vector(v_perp1)
    v_perp2_normalized = normalize_vector(v_perp2)

    return v_perp1_normalized, v_perp2_normalized
	

In [13]:
alpha = 0.0
theta = 0.0
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.1
proximity_threshold = 1.0
ncuts_threshold = 0.09
        
out_name = 'sam_only' + str(SEQUENCE_NUM) 
# Generate 30 different colors
 

COLORS = generate_random_colors(400)
COLORS = [(col[0]/255.,col[1]/255.,col[2]/255.) for col in COLORS]


out_kitti = out_chunks + 'out_kitti' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_kitti) == True : 
        shutil.rmtree(out_kitti)

os.makedirs(out_kitti)
        
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'
'''
if os.path.exists(out_kitti_instance) == True : 
        shutil.rmtree(out_kitti_instance)
os.makedirs(out_kitti_instance)
'''


out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'
'''
if os.path.exists(out_kitti_semantic) == True : 
        shutil.rmtree(out_kitti_semantic)
os.makedirs(out_kitti_semantic)
'''

limit = -1 ##use this for experiments to run limit chunks numberss


patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
semantics = np.hstack((kitti_labels_orig['seg_nonground'].reshape(-1,),kitti_labels_orig['seg_ground'].reshape(-1,)))

instances = np.hstack((kitti_labels_orig['instance_nonground'],kitti_labels_orig['instance_ground']))
                
merge_pcd = o3d.geometry.PointCloud()    
                
print("total sequence number",len(center_ids))
for sequence in range(0,len(center_ids)):
        if NCUT_ground == False : 
                
                merged_chunk,file_name, pcd_chunk, pcd_chunk_ground,inliers, inliers_ground = ncuts_chunk(dataset,list(indices),pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,list(sampled_indices_global),
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                        patchwise_indices=patchwise_indices,ncuts_threshold=ncuts_threshold)
                
                ground_out,file_name = ncuts_chunk(dataset,list(indices_ground),pcd_chunk_ground,None,
                        pcd_ground_chunks_major_downsampling,
                        pcd_ground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,list(sampled_indices_global),
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=1.0,beta=0.0,gamma=1.0,theta=1.0,
                        proximity_threshold=1.0,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices,ncuts_threshold=0.001,use_z=False)
                #o3d.visualization.draw_geometries([pcd_chunk + ground_out])
                
                #o3d.visualization.draw_geometries([ground_out])
                
                
                seg_ground = kitti_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                inst_ground = kitti_labels['ground']['instance'][sequence][inliers][inliers_ground]
                
                file_name = str(center_ids[sequence]).zfill(6) + '.pcd'

                kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['semantic'][sequence].reshape(-1,),
                                        colors=COLORS,gt_labels=semantics,semantics=True
                                        )
                
                kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,inst_ground.reshape(-1,),
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_semantic_ground = color_pcd_by_labels(pcd_chunk_ground,seg_ground.reshape(-1,),
                                        colors=COLORS,gt_labels=semantics,semantics=True)
                
                #o3d.visualization.draw_geometries([kitti_chunk_instance + kitti_chunk_instance_ground])
                #o3d.visualization.draw_geometries([pcd_chunk + pcd_chunk_ground])
                semantic_labels = np.hstack(( kitti_labels['nonground']['semantic'][sequence].reshape(-1,),seg_ground.reshape(-1,)))
                np.savez(out_kitti_semantic + file_name.split('.')[0] + '.npz',labels=semantic_labels)
                
                o3d.io.write_point_cloud(out_folder_ncuts + file_name, pcd_chunk + ground_out , write_ascii=False, compressed=False, print_progress=False)
                o3d.io.write_point_cloud(out_kitti_instance + file_name, kitti_chunk_instance + kitti_chunk_instance_ground, write_ascii=False, compressed=False, print_progress=False)
                #o3d.io.write_point_cloud(out_kitti_semantic + file_name, kitti_chunk + kitti_chunk_semantic_ground, write_ascii=False, compressed=False, print_progress=False)
                
                


        
        
        

total sequence number 30
Start of sequence 0
7776 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 58 cut regions
Ratio of points in top 3 groups: 0.25655864197530864
Start of sequence 0
5294 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 262 cut regions
Ratio of points in top 3 groups: 0.3917642614280317
Start of sequence 1
7216 points in downsampled chunk (major)


In [ ]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds




def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

In [ ]:
from tqdm import tqdm 
def merge_chunks_unite_instances2(chunks: list, icp=False):
    merge = o3d.geometry.PointCloud()
    chunk_means = [np.mean(np.asarray(chunk.points), axis=0) for chunk in chunks]

    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in tqdm(chunks[1:]):

        new_chunk_center = np.asarray(new_chunk.points)
        
        x,y,z = new_chunk_center[:,0].mean(),new_chunk_center[:,1].mean(),new_chunk_center[:,2].mean()
        
        
        '''
        pcd_tree = o3d.geometry.KDTreeFlann(merge)
        query_point = np.array([x, y, z])  # Replace x, y, z with your point's coordinates
        distance_threshold = 35.0  # Adjust this value to your desired distance threshold
        
        
        
        [k, idx, _] = pcd_tree.search_radius_vector_3d(query_point, distance_threshold)

        # Extract the points within the specified distance
        if k > 0:
            points_within_distance = np.asarray(merge.points)[idx]
        else:
            points_within_distance = np.array([])

        extracted_pcd = o3d.geometry.PointCloud()
        extracted_pcd.points = o3d.utility.Vector3dVector(points_within_distance)
        extracted_pcd.colors = o3d.utility.Vector3dVector(np.asarray(merge.colors)[idx])
        '''
        side_length = 40 
        center_point = np.array([x, y, z])  # Replace x, y, z with your point's coordinates
        half_side = side_length / 2.0
        min_bound = center_point - half_side
        max_bound = center_point + half_side
        aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)

        # Crop the point cloud
        extracted_pcd = merge.crop(aabb)

        
        points_1 = np.asarray(extracted_pcd.points)
        points_2 = np.asarray(new_chunk.points)

        colors_1 = np.asarray(extracted_pcd.colors)
        colors_2 = np.asarray(new_chunk.colors)

        unique_colors_1 = np.unique(colors_1, axis=0)
        unique_colors_2 = np.unique(colors_2, axis=0)

        instance2point_1 = {}
        for i in range(unique_colors_1.shape[0]):
            if not np.all(unique_colors_1[i] == 0.0): # Streets are black
                instance2point_1[i] = {}
                inds = np.where(np.all(colors_1 == unique_colors_1[i], axis=1))[0]
                instance2point_1[i]["points"] = points_1[inds]
                instance2point_1[i]["inds"] = inds

        instance2point_2 = {}
        for i in range(unique_colors_2.shape[0]):
            if not np.all(unique_colors_2[i] == 0.0): # Streets are black
                instance2point_2[i] = {}
                inds = np.where(np.all(colors_2 == unique_colors_2[i], axis=1))[0]
                instance2point_2[i]["points"] = points_2[inds]
                instance2point_2[i]["inds"] = inds
        
        id_pairs_iou = []
        for id_1, entries_1 in instance2point_1.items():
            points1 = entries_1["points"]
            min_bound = np.min(points1, axis=0)
            max_bound = np.max(points1, axis=0)
            association = []
            for id_2, entries_2 in instance2point_2.items():
                points2 = entries_2["points"]
                intersection = np.where(np.all(points2 >= min_bound, axis=1) & np.all(points2 <= max_bound, axis=1))[0].shape[0]
                if intersection > 0:
                    union = len(np.unique(np.concatenate((points1, points2))))
                    iou = float(intersection) / float(union)
                    if iou > 0.01:
                        association.append((id_2, iou))
            if len(association) != 0:
                for (association_id, iou) in association:
                    id_pairs_iou.append((id_1, (association_id, iou)))
        
        ids_chunk_1 = []
        ids_chunk_2 = []
        ious = []
        for id1, (id2, iou) in id_pairs_iou:
            if id2 not in ids_chunk_2:
                ids_chunk_1.append(id1)
                ids_chunk_2.append(id2)
                ious.append(iou)
            else:
                i = ids_chunk_2.index(id2)
                if iou > ious[i]:
                    ious[i] = iou
                    ids_chunk_1[i] = id1

        for id1, id2 in zip(ids_chunk_1, ids_chunk_2):
            inds2 = instance2point_2[id2]["inds"]
            colors_2[inds2] = unique_colors_1[id1]

        new_chunk_recolored = o3d.geometry.PointCloud()
        new_chunk_recolored.points = new_chunk.points
        new_chunk_recolored.colors = o3d.utility.Vector3dVector(colors_2)
        last_chunk = new_chunk_recolored

        merge += new_chunk_recolored
        merge.remove_duplicated_points()

    return merge

In [ ]:
out_kitti = out_chunks + 'out_kitti' + str(SEQUENCE_NUM) + '/'
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'
out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'

point_clouds = get_merge_pcds(out_folder_ncuts)[:-1]
#point_clouds_kitti = get_merge_pcds(out_kitti)[:-1]
point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance)[:-1]
#point_clouds_kitti_semantic = get_merge_pcds(out_kitti_semantic)[:-1]
merge = merge_chunks_unite_instances(point_clouds)


#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)

#merge_kitti = merge_unite_gt(point_clouds_kitti)
merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
#merge_kitti_semantic = merge_unite_gt(point_clouds_kitti_semantic)

#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_semantic7.pcd", merge_kitti_semantic, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + out_name + str(SEQUENCE_NUM) + ".pcd", merge, write_ascii=False, compressed=False, print_progress=False)
print(merge_kitti_instance)
print(merge)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance" + str(SEQUENCE_NUM) +  ".pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)


In [ ]:

from tqdm import tqdm 

merge = o3d.io.read_point_cloud(out_folder + out_name + str(SEQUENCE_NUM) +  ".pcd")
unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + "merge_part_kitti_instance" + str(SEQUENCE_NUM) +  ".pcd")
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
print(merge_kitti_instance)
def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


from concurrent.futures import ThreadPoolExecutor

def process_batch(unique_pred, preds, labels, gt_idcs, threshold, new_ncuts_labels):
    pred_idcs = np.where(preds == unique_pred)[0]
    cur_intersect = np.sum(np.isin(pred_idcs, gt_idcs))
    if cur_intersect > threshold * len(pred_idcs): 
        new_ncuts_labels[pred_idcs] = 0

def remove_semantics(labels, preds, threshold=0.8, num_threads=None):
    gt_idcs = np.where(labels == 0)[0]
    new_ncuts_labels = preds.copy()
    unique_preds = np.unique(preds)
    
    if num_threads is None:
        num_threads = min(len(unique_preds), 8)  # Default to 8 threads if not specified
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for i in tqdm(unique_preds):
            futures.append(executor.submit(process_batch, i, preds, labels, gt_idcs, threshold, new_ncuts_labels))
        
        # Wait for all tasks to complete
        for future in tqdm(futures, total=len(futures), desc="Processing"):
            future.result()  # Get the result to catch any exceptions
        
    return new_ncuts_labels

print('remove semantics')
new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)


metrics_ncuts = Metrics(name='ncuts')

metrics_ncuts.update_stats(labels_ncuts,new_ncuts_labels,labels_kitti)

#merge_vis = color_pcd_by_labels(merge,new_ncuts_labels)
#o3d.visualization.draw_geometries([merge_vis])
